In [1]:
from sklearn.datasets import load_iris
import torch
from torch import nn

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
device

device(type='cuda')

In [15]:
X,y = load_iris(return_X_y=True)

In [16]:
X = torch.from_numpy(X).type(torch.float).to(device)
y = torch.from_numpy(y).type(torch.long).to(device)

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=26)

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([120, 4]),
 torch.Size([30, 4]),
 torch.Size([120]),
 torch.Size([30]))

In [19]:
X_train.device, X_test.device, y_train.device, y_test.device

(device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0))

In [38]:
# model
import torch
from torch import nn

class FlowerClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.l1 = nn.Linear(in_features=4, out_features=10)
        self.l2 = nn.Linear(in_features=10, out_features=3)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x

In [39]:
model0 = FlowerClassifier()

In [40]:
model0

FlowerClassifier(
  (l1): Linear(in_features=4, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=3, bias=True)
  (relu): ReLU()
)

In [41]:
next(model0.parameters()).device

device(type='cpu')

In [42]:
# setting it to device
model0.to(device)

FlowerClassifier(
  (l1): Linear(in_features=4, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=3, bias=True)
  (relu): ReLU()
)

In [43]:
model0(X_train)

tensor([[-0.3128, -0.1016, -1.0153],
        [-0.3437, -0.1155, -0.9149],
        [-0.3495, -0.0575, -1.0823],
        [-0.3349,  0.0024, -1.2656],
        [-0.3998, -0.2093, -0.5718],
        [-0.3803, -0.2257, -0.4988],
        [-0.3931, -0.1877, -0.7145],
        [-0.3990, -0.0580, -0.9692],
        [-0.3924, -0.2241, -0.4765],
        [-0.3867,  0.0272, -1.2735],
        [-0.4133, -0.0533, -0.9438],
        [-0.3723, -0.0917, -0.9183],
        [-0.3930, -0.0962, -0.9313],
        [-0.3761, -0.0066, -1.0388],
        [-0.3677,  0.0218, -1.2131],
        [-0.3584, -0.2102, -0.6866],
        [-0.4362, -0.0168, -1.1176],
        [-0.3723,  0.0475, -1.3173],
        [-0.3538, -0.0093, -1.1675],
        [-0.3369, -0.0824, -1.1630],
        [-0.3834, -0.0936, -0.9168],
        [-0.4062, -0.1910, -0.6571],
        [-0.4254, -0.1938, -0.6167],
        [-0.3940, -0.1948, -0.6966],
        [-0.3028, -0.2640, -0.4692],
        [-0.3830, -0.2236, -0.4993],
        [-0.3638, -0.2214, -0.6019],
 

In [44]:
loss_fn = nn.CrossEntropyLoss()
optimizer_fn = torch.optim.Adam(params=model0.parameters(), lr=0.1)


In [45]:
## test model
# c = model0(X_train)

In [46]:
# training loop

epoch = 1000
for e in range(epoch):
    model0.train()
    
    y_logits = model0(X_train)
#     print(y_logits)
    loss = loss_fn(y_logits, y_train)
    optimizer_fn.zero_grad()
    loss.backward()
    optimizer_fn.step()
    model0.eval()
    if e%100 == 0:
        with torch.inference_mode():
            print(e, "loss:",loss.cpu().detach().numpy())

0 loss: 1.2484545
100 loss: 0.07067658
200 loss: 0.06085237
300 loss: 0.055808526
400 loss: 0.052590385
500 loss: 0.05063651
600 loss: 0.04956914
700 loss: 0.048946932
800 loss: 0.052215498
900 loss: 0.05132576


In [47]:
y_test.cpu().detach().numpy()

array([1, 1, 0, 0, 2, 1, 0, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 1, 2, 0, 0, 2,
       2, 2, 2, 2, 1, 0, 0, 2])

In [48]:
from sklearn.metrics import accuracy_score
import numpy as np
y_preds = []
for xt in X_test:
    ypreds = model0(xt)
    y_preds.append(ypreds.argmax().item())
# y_preds = np.array(y_preds)

np.array(y_preds)

array([1, 1, 0, 0, 2, 1, 0, 2, 2, 2, 0, 1, 1, 0, 1, 0, 2, 1, 2, 0, 0, 2,
       2, 2, 2, 2, 1, 0, 0, 2])

In [49]:
accuracy_score(y_test.cpu().detach().numpy(), y_preds)

1.0